In [ ]:
# ==========================================
# BanglaT5 Fine-Tuning (Dialect → Standard Bangla)
# ==========================================

!pip install -q transformers[sentencepiece] datasets evaluate sacrebleu torch pandas openpyxl accelerate

import pandas as pd
import numpy as np
import torch
import evaluate
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
)
from google.colab import drive

# ==========================================
#  Mount Google Drive & Load Dataset
# ==========================================
drive.mount('/content/drive')

try:
    df = pd.read_excel("/content/drive/MyDrive/bangla_dialect_aligned_18920.xlsx")
    print(f" Dataset loaded successfully. Total rows: {len(df)}")
except FileNotFoundError:
    print(" File not found — using a small demo dataset instead.")
    df = pd.DataFrame({
        'Standard_Bangla': ["সে স্কুলে যায়।", "তুমি কোথায় যাও?", "আমি বই পড়ি।"],
        'Chittagong': ["হেই ইস্কুলে যায়।", "তুই কই যাও?", "আঁই বই পড়মু।"]
    })

# Select which dialect(s) to train
DIALECTS_TO_TRAIN = ['Barisal', 'Sylhet']

# ==========================================
# Load Tokenizer & Model
# ==========================================
MODEL_NAME = "csebuetnlp/banglat5"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
print(f"\n Tokenizer for '{MODEL_NAME}' loaded.\n")

# ==========================================
# 🧩 Dataset Preparation
# ==========================================
def create_dataset_dict(dialect_col):
    subset_df = df[['Standard_Bangla', dialect_col]].dropna()
    subset_df = subset_df.rename(columns={'Standard_Bangla': 'target', dialect_col: 'source'})
    print(f"Dialect '{dialect_col}' samples: {len(subset_df)}")

    hf_dataset = Dataset.from_pandas(subset_df)
    train_test_split = hf_dataset.train_test_split(test_size=0.2, seed=42)
    test_val_split = train_test_split['test'].train_test_split(test_size=0.5, seed=42)

    return DatasetDict({
        'train': train_test_split['train'],
        'validation': test_val_split['train'],
        'test': test_val_split['test']
    })

# ==========================================
#  Tokenization
# ==========================================
def tokenize_and_prepare_datasets(dataset_dict):
    def tokenize_fn(examples):
        model_inputs = tokenizer(
            examples["source"], max_length=64, truncation=True, padding="max_length"
        )
        labels = tokenizer(
            text_target=examples["target"], max_length=64, truncation=True, padding="max_length"
        )
        # Mask out padding tokens
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label]
            for label in labels["input_ids"]
        ]
        model_inputs["labels"] = labels["input_ids"]
        return model_inputs

    tokenized_datasets = dataset_dict.map(tokenize_fn, batched=True)
    print(" Tokenization complete.")
    return tokenized_datasets

# ==========================================
#  Train & Evaluate Function
# ==========================================
def train_and_evaluate(dialect_name, train_ds, val_ds, test_ds):
    output_dir = f"/content/drive/MyDrive/Bangla_Dialect_Models/BanglaT5-{dialect_name.lower()}"

    model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)
    data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)
    bleu_metric = evaluate.load("sacrebleu")

    def compute_metrics(eval_pred):
        preds, labels = eval_pred
        # Convert predictions to the appropriate integer type if necessary
        if isinstance(preds, np.ndarray) and preds.dtype != np.int64:
            preds = preds.astype(np.int64)
        elif torch.is_tensor(preds):
            preds = preds.to(torch.int64)

        # Replace out-of-vocabulary token IDs with padding token ID
        preds = np.where((preds >= 0) & (preds < tokenizer.vocab_size), preds, tokenizer.pad_token_id)

        decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

        decoded_preds = [p.strip() for p in decoded_preds]
        decoded_labels = [[l.strip()] for l in decoded_labels]

        result = bleu_metric.compute(predictions=decoded_preds, references=decoded_labels)
        return {"bleu": result["score"]}

    training_args = Seq2SeqTrainingArguments(
        output_dir=output_dir,
        learning_rate=3e-4,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        num_train_epochs=5,
        weight_decay=0.01,
        predict_with_generate=True,
        fp16=False,                             # disable to avoid silent overflow
        logging_strategy="steps",
        logging_steps=50,
        eval_strategy="steps",
        eval_steps=200,
        save_steps=500,
        save_total_limit=2,
        generation_max_length=64,
        generation_num_beams=4,
        report_to="none",
    )

    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

    print(f"\n Training BanglaT5 for dialect: {dialect_name}")
    trainer.train()

    print(f"\n Evaluating model on test set for '{dialect_name}'...")
    results = trainer.predict(test_ds)
    bleu_score = results.metrics.get('test_bleu', 0.0)
    print(f" Test BLEU Score for {dialect_name}: {bleu_score:.2f}")

    print("\n Example Outputs:")
    pred_ids = results.predictions
    # Convert predictions to the appropriate integer type if necessary
    if isinstance(pred_ids, np.ndarray) and pred_ids.dtype != np.int64:
      pred_ids = pred_ids.astype(np.int64)
    elif torch.is_tensor(pred_ids):
      pred_ids = pred_ids.to(torch.int64)

    # Replace out-of-vocabulary token IDs with padding token ID for example outputs
    pred_ids = np.where((pred_ids >= 0) & (pred_ids < tokenizer.vocab_size), pred_ids, tokenizer.pad_token_id)


    preds = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    for i in range(min(3, len(preds))):
        print(f"Dialect:   {test_ds[i]['source']}")
        print(f"Standard:  {test_ds[i]['target']}")
        print(f"Predicted: {preds[i]}\n")

# ==========================================
#  Run Fine-Tuning
# ==========================================
for dialect in DIALECTS_TO_TRAIN:
    dataset_dict = create_dataset_dict(dialect)
    tokenized_data = tokenize_and_prepare_datasets(dataset_dict)
    train_and_evaluate(
        dialect,
        tokenized_data["train"],
        tokenized_data["validation"],
        tokenized_data["test"]
    )

print("\n All training completed! Models saved to your Google Drive.")

Mounted at /content/drive
 Dataset loaded successfully. Total rows: 3480


tokenizer_config.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.11M [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

...

/tmp/ipython-input-1481422616.py:138: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(



 Training BanglaT5 for dialect: Sylhet


Step,Training Loss,Validation Loss,Bleu
200,3.793900,2.388562,21.209023
400,2.629600,1.637667,36.640590
600,2.063600,1.318723,40.996558
800,1.446900,1.189013,44.937054
1000,1.354800,1.152171,46.065186
1200,1.210300,1.119686,47.075142
1400,1.118500,1.058917,50.120182
1600,0.913500,1.063503,50.032885
1800,1.028400,1.011335,50.674826
2000,0.910500,0.998109,51.417693



 Evaluating model on test set for 'Sylhet'...


 Test BLEU Score for Sylhet: 49.80

 Example Outputs:
Dialect:   আফনে কিতা আমার প্রশ্ন'র জবাব দিতে ফারবা নি?
Standard:  আপনি কি আমার প্রশ্নের জবাব দিতে পারেন?
Predicted: আপনি কি আমার প্রশ্নের জবাব দিতে পারেন?

Dialect:   আকাশ ওর নীল রঙটা অসাধারণ
Standard:  আকাশের নীল রঙটি অসাধারণ
Predicted: আকাশের নীল রঙটা অসাধারণ

Dialect:   ছোট বইনে চিল্লাইয়া উঠলো, দারুণ
Standard:  ছোটো বোন চিৎকার দিয়ে উঠল, দারুণ
Predicted: ছোট বোন চিৎকার করে উঠলো, অসাধারণ


 All training completed! Models saved to your Google Drive.


In [ ]:
# ==========================================
# BanglaT5 Fine-Tuning (Dialect → Standard Bangla)
# ==========================================

!pip install -q transformers[sentencepiece] datasets evaluate sacrebleu torch pandas openpyxl accelerate

import pandas as pd
import numpy as np
import torch
import evaluate
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
)
from google.colab import drive

# ==========================================
#  Mount Google Drive & Load Dataset
# ==========================================
drive.mount('/content/drive')

try:
    df = pd.read_excel("/content/drive/MyDrive/bangla_dialect_aligned_18920.xlsx")
    print(f" Dataset loaded successfully. Total rows: {len(df)}")
except FileNotFoundError:
    print(" File not found — using a small demo dataset instead.")
    df = pd.DataFrame({
        'Standard_Bangla': ["সে স্কুলে যায়।", "তুমি কোথায় যাও?", "আমি বই পড়ি।"],
        'Chittagong': ["হেই ইস্কুলে যায়।", "তুই কই যাও?", "আঁই বই পড়মু।"]
    })

# Select which dialect(s) to train
DIALECTS_TO_TRAIN = ['Noakhali', 'Mymensingh']

# ==========================================
# Load Tokenizer & Model
# ==========================================
MODEL_NAME = "csebuetnlp/banglat5"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
print(f"\n Tokenizer for '{MODEL_NAME}' loaded.\n")

# ==========================================
# 🧩 Dataset Preparation
# ==========================================
def create_dataset_dict(dialect_col):
    subset_df = df[['Standard_Bangla', dialect_col]].dropna()
    subset_df = subset_df.rename(columns={'Standard_Bangla': 'target', dialect_col: 'source'})
    print(f"Dialect '{dialect_col}' samples: {len(subset_df)}")

    hf_dataset = Dataset.from_pandas(subset_df)
    train_test_split = hf_dataset.train_test_split(test_size=0.2, seed=42)
    test_val_split = train_test_split['test'].train_test_split(test_size=0.5, seed=42)

    return DatasetDict({
        'train': train_test_split['train'],
        'validation': test_val_split['train'],
        'test': test_val_split['test']
    })

# ==========================================
#  Tokenization
# ==========================================
def tokenize_and_prepare_datasets(dataset_dict):
    def tokenize_fn(examples):
        model_inputs = tokenizer(
            examples["source"], max_length=64, truncation=True, padding="max_length"
        )
        labels = tokenizer(
            text_target=examples["target"], max_length=64, truncation=True, padding="max_length"
        )
        # Mask out padding tokens
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label]
            for label in labels["input_ids"]
        ]
        model_inputs["labels"] = labels["input_ids"]
        return model_inputs

    tokenized_datasets = dataset_dict.map(tokenize_fn, batched=True)
    print(" Tokenization complete.")
    return tokenized_datasets

# ==========================================
#  Train & Evaluate Function
# ==========================================
def train_and_evaluate(dialect_name, train_ds, val_ds, test_ds):
    output_dir = f"/content/drive/MyDrive/Bangla_Dialect_Models/BanglaT5-{dialect_name.lower()}"

    model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)
    data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)
    bleu_metric = evaluate.load("sacrebleu")

    def compute_metrics(eval_pred):
        preds, labels = eval_pred
        # Convert predictions to the appropriate integer type if necessary
        if isinstance(preds, np.ndarray) and preds.dtype != np.int64:
            preds = preds.astype(np.int64)
        elif torch.is_tensor(preds):
            preds = preds.to(torch.int64)

        # Replace out-of-vocabulary token IDs with padding token ID
        preds = np.where((preds >= 0) & (preds < tokenizer.vocab_size), preds, tokenizer.pad_token_id)

        decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

        decoded_preds = [p.strip() for p in decoded_preds]
        decoded_labels = [[l.strip()] for l in decoded_labels]

        result = bleu_metric.compute(predictions=decoded_preds, references=decoded_labels)
        return {"bleu": result["score"]}

    training_args = Seq2SeqTrainingArguments(
        output_dir=output_dir,
        learning_rate=3e-4,                     # slightly higher for better convergence
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        num_train_epochs=5,
        weight_decay=0.01,
        predict_with_generate=True,
        fp16=False,                             # disable to avoid silent overflow
        logging_strategy="steps",
        logging_steps=50,
        eval_strategy="steps",
        eval_steps=200,
        save_steps=500,
        save_total_limit=2,
        generation_max_length=64,
        generation_num_beams=4,
        report_to="none",
    )

    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

    print(f"\n Training BanglaT5 for dialect: {dialect_name}")
    trainer.train()

    print(f"\n Evaluating model on test set for '{dialect_name}'...")
    results = trainer.predict(test_ds)
    bleu_score = results.metrics.get('test_bleu', 0.0)
    print(f" Test BLEU Score for {dialect_name}: {bleu_score:.2f}")

    print("\n Example Outputs:")
    pred_ids = results.predictions
    # Convert predictions to the appropriate integer type if necessary
    if isinstance(pred_ids, np.ndarray) and pred_ids.dtype != np.int64:
      pred_ids = pred_ids.astype(np.int64)
    elif torch.is_tensor(pred_ids):
      pred_ids = pred_ids.to(torch.int64)

    # Replace out-of-vocabulary token IDs with padding token ID for example outputs
    pred_ids = np.where((pred_ids >= 0) & (pred_ids < tokenizer.vocab_size), pred_ids, tokenizer.pad_token_id)


    preds = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    for i in range(min(3, len(preds))):
        print(f"Dialect:   {test_ds[i]['source']}")
        print(f"Standard:  {test_ds[i]['target']}")
        print(f"Predicted: {preds[i]}\n")

# ==========================================
#  Run Fine-Tuning
# ==========================================
for dialect in DIALECTS_TO_TRAIN:
    dataset_dict = create_dataset_dict(dialect)
    tokenized_data = tokenize_and_prepare_datasets(dataset_dict)
    train_and_evaluate(
        dialect,
        tokenized_data["train"],
        tokenized_data["validation"],
        tokenized_data["test"]
    )

print("\n All training completed! Models saved to your Google Drive.")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 6.9 MB/s eta 0:00:00
Mounted at /content/drive
 Dataset loaded successfully. Total rows: 3480


tokenizer_config.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.11M [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

...

/tmp/ipython-input-3704737.py:138: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(



 Training BanglaT5 for dialect: Mymensingh


Step,Training Loss,Validation Loss,Bleu
200,2.291800,1.227936,52.082886
400,1.455900,0.685322,68.820078
600,0.892600,0.631858,72.238863
800,0.608600,0.606383,73.395013
1000,0.609600,0.562501,75.346593
1200,0.432200,0.560546,76.234691
1400,0.439900,0.545681,76.797089
1600,0.382600,0.531948,78.091593
1800,0.334300,0.541260,78.681931
2000,0.274300,0.561232,77.812434



 Evaluating model on test set for 'Mymensingh'...


 Test BLEU Score for Mymensingh: 77.53

 Example Outputs:
Dialect:   যাওনের লগে লগে কী একটা শব্দ করল
Standard:  যাওয়ার সাথে সাথে কী একটা শব্দ করল
Predicted: যাওয়ার সঙ্গে সঙ্গে কী একটা শব্দ করল

Dialect:   তুমি কিতা আমারে এই কামডা কইরা দিতা ফারবা?
Standard:  তুমি কি আমাকে এই কাজটি করে দিতে পারবে?
Predicted: তুমি কি আমাকে এই কাজটি করে দিতে পারবে?

Dialect:   তর পরিক্ষা কবে ?
Standard:  তোর পরিক্ষা কবে ?
Predicted: তোর পরিক্ষা কবে ?


 All training completed! Models saved to your Google Drive.
